# Keras輸入資料的方法  
使用`fit()` method輸入資料有以下幾種方法：  
* `fit(x=train_x, y=train_y, ...)`：所有資料會載入GPU RAM中，若資料龐大，則必須使用其他方式載入。  
* `fit(keras.utils.Sequence, ...)`：使用Python generato載入資料，可以解決無法一次將所有資料載入GPU RAM中的問題。  
* `fit(tf.Dataset, ...)`：TensorFlow提供的載入資料方式，也可以解決資料過大的問題。  

這邊主要說明`Sequence`的方式，更多資訊參閱[Training & evaluation with the built-in methods](https://keras.io/guides/training_with_built_in_methods/)。

In [1]:
# 載入所需lib
import numpy as np
import math
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.2.0


Keras Sequence主要要寫`__getitme__`與`__len__`兩個method，若有需要打散data可在`on_epoch_end` method中增加打散方式。  
以下是範例：

In [2]:
#MNIST 手寫資料
class MNIST_Sequence(tf.keras.utils.Sequence):
    
    # 初始化一些參數資料，假設輸入是圖像路徑列表，可在__getitem__中寫入讀取方式以及處理
    # 因為使用MNIST，所以此處直接回傳批次(batch)
    def __init__(self, x_train, y_train, batch_size):
        self.x, self.y = x_train, y_train
        self.batch_size = batch_size
        
    # 計算一次疊代有多少個batch
    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)
    
    # 每一個batch生成的資料在這處理
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size: (idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size: (idx + 1) * self.batch_size]
        
        batch_x = batch_x.astype('float32') / 255
        batch_y = batch_y.astype('float32')
        
        batch_x = np.expand_dims(batch_x, -1)
        batch_y = np.expand_dims(batch_y, -1)
        
        return batch_x, batch_y
    
    # 每個epoch結束後打亂資料
    def on_epoch_end(self):
        # 存放此次打散的參數
        rng_state = np.random.get_state()
        np.random.shuffle(self.x)
        # 還原打散前的狀態，在打散y，則打散就可以對應
        np.random.set_state(rng_state)
        np.random.shuffle(self.y)

In [3]:
#download MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print('x_train\nshape:{},data type:{}'.format(x_train.shape, x_train.dtype))
print('y_train\nshape:{},data type:{}'.format(y_train.shape, y_train.dtype))
print('x_test\nshape:{},data type:{}'.format(x_test.shape, x_test.dtype))
print('y_test\nshape:{},data type:{}'.format(y_test.shape, y_test.dtype))

x_train
shape:(60000, 28, 28),data type:uint8
y_train
shape:(60000,),data type:uint8
x_test
shape:(10000, 28, 28),data type:uint8
y_test
shape:(10000,),data type:uint8


In [4]:
#input layer(使用Conv2D需要3維(h, w, c)，所以上方只有(28, 28)需做處理變成(28, 28, 1)
inputs = tf.keras.Input(shape=(28, 28, 1))

# model layer
conv_1 = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu')
max_pool_1 = tf.keras.layers.MaxPooling2D()
conv_2 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu')
max_pool_2 = tf.keras.layers.MaxPooling2D()
flatten = tf.keras.layers.Flatten()
drop = tf.keras.layers.Dropout(0.5)
output = tf.keras.layers.Dense(10, activation='softmax')

# path
x = conv_1(inputs)
x = max_pool_1(x)
x = conv_2(x)
x = max_pool_2(x)
x = flatten(x)
x = drop(x)
x = output(x)

model = tf.keras.Model(inputs=inputs, outputs=x)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0     

In [5]:
#compile model
#sparse_categorical_crossentropy:可以直接對應數字，而不用轉換成one-hot encoding
#有些內建的function可以直接輸入名稱帶入，不然就得放置相應的function
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['sparse_categorical_accuracy']
)
history = model.fit(MNIST_Sequence(x_train, y_train, 128), epochs=10)

Epoch 1/10
469/469 [==============================] - 2s 4ms/step - loss: 0.3456 - sparse_categorical_accuracy: 0.8945
Epoch 2/10
469/469 [==============================] - 2s 4ms/step - loss: 0.1074 - sparse_categorical_accuracy: 0.9669
Epoch 3/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0821 - sparse_categorical_accuracy: 0.9744
Epoch 4/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0679 - sparse_categorical_accuracy: 0.9795
Epoch 5/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0605 - sparse_categorical_accuracy: 0.9817
Epoch 6/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0548 - sparse_categorical_accuracy: 0.9832
Epoch 7/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0510 - sparse_categorical_accuracy: 0.9839
Epoch 8/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0456 - sparse_categorical_accuracy: 0.9862
Epoch 9/10
469/469 [============================

`model.evaluate()`也是相同方法：

In [6]:
score = model.evaluate(MNIST_Sequence(x_test, y_test, 128))
print('test loss:', score[0])
print('test accuracy', score[1])

79/79 [==============================] - 0s 4ms/step - loss: 0.0262 - sparse_categorical_accuracy: 0.9905
test loss: 0.026222582906484604
test accuracy 0.9904999732971191


## **總結**  
此處演示`keras.utils.Sequence`的用法，當想使用`fit()`但是資料龐大無法放入GPU時的解決方法。  
`__init__`、`__len__`、`__getitem__`、`on_epoch_end`，皆有不同的功能，根據需要進行修改。